# Optimization using PSO

##  Import libraries

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from pyswarm import pso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
# Load dataset
data = pd.read_csv(r"C:\Users\amnsh\OneDrive\Desktop\DAA\Dataset_cia2\Bank_Personal_Loan_Modelling.csv")

In [ ]:
# Split dataset into input and output variables
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale input data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Neural network architecture

In [ ]:
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    return model

## Function for PSO to train neural network

In [ ]:
def train_model(params):
    model = create_model(X_train.shape[1])
    model.set_weights(params)
    history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0, validation_split=0.2)
    val_loss = history.history['val_loss'][-1]
    return val_loss

In [ ]:
# Define bounds for PSO parameters
bounds = [(-1, 1)] * sum([coef.size for coef in create_model(X_train.shape[1]).get_weights()])

In [ ]:
# Train neural network using PSO
best_weights, best_cost = pso(train_model, bounds, swarmsize=10, maxiter=10, minstep=1e-6, minfunc=1e-6)

In [ ]:
# Create final model with best weights
final_model = create_model(X_train.shape[1])
final_model.set_weights(best_weights)

In [ ]:
# Evaluate final model on testing set
y_pred = final_model.predict_classes(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))